In [ ]:
from abc import ABC, abstractmethod


class Book:
    def __init__(self, book_id: str, title: str, author: str):
        self.book_id = book_id
        self.title = title
        self.author = author
        self.__available = True  # private attribute (encapsulation)

    # property for controlled access to availability
    @property
    def available(self) -> bool:
        return self.__available

    @available.setter
    def available(self, value: bool):
        self.__available = bool(value)

    def display_info(self):
        status = "Available" if self.__available else "Issued"
        print(f"[{self.book_id}] {self.title} by {self.author} — {status}")

    def mark_issued(self):
        self.__available = False

    def mark_returned(self):
        self.__available = True



In [8]:
class Person(ABC):
    """Abstract base class for people (members)."""
    def __init__(self, member_id: str, name: str):
        self._member_id = member_id         # protected
        self.name = name

    @property
    def member_id(self):
        return self._member_id

    @abstractmethod
    def display_member_info(self):
        pass


class Member(Person):
    """Base Member class with basic issue/return functionality."""
    BORROW_LIMIT = 0  # override in subclasses

    def __init__(self, member_id: str, name: str):
        super().__init__(member_id, name)
        self.issued_books = []  # list of Book objects

    def can_borrow(self) -> bool:
        return len(self.issued_books) < self.BORROW_LIMIT

    def issue_book(self, book: Book) -> bool:
        """Issue book to member if possible. Returns True on success."""
        if not book.available:
            # book already issued
            return False
        if not self.can_borrow():
            # reached borrowing limit
            return False
        book.mark_issued()
        self.issued_books.append(book)
        return True

    def return_book(self, book: Book) -> bool:
        """Return book from member. Returns True on success."""
        if book in self.issued_books:
            self.issued_books.remove(book)
            book.mark_returned()
            return True
        return False

    def display_member_info(self):
        print(f"Member ID: {self.member_id} | Name: {self.name}")
        print(f"Issued books ({len(self.issued_books)}):")
        for b in self.issued_books:
            print(f"  - [{b.book_id}] {b.title}")


In [9]:
class StudentMember(Member):
    BORROW_LIMIT = 3

    def display_member_info(self):
        # polymorphism: different message for students
        print(f"Student Member — ID: {self.member_id}, Name: {self.name}")
        print(f"(Student borrow limit: {self.BORROW_LIMIT})")
        super().display_member_info()

In [10]:
class TeacherMember(Member):
    BORROW_LIMIT = 5

    def display_member_info(self):
        # polymorphism: different message for teachers
        print(f"Teacher Member — ID: {self.member_id}, Name: {self.name}")
        print(f"(Teacher borrow limit: {self.BORROW_LIMIT})")
        super().display_member_info()

In [11]:
class Library:
    """Library manages books and members and handles issues/returns."""
    def __init__(self):
        # use dicts for O(1) lookup by id
        self.books = {}    # book_id -> Book
        self.members = {}  # member_id -> Member

    def add_book(self, book: Book):
        self.books[book.book_id] = book

    def register_member(self, member: Member):
        self.members[member.member_id] = member

    def issue_book(self, member_id: str, book_id: str) -> bool:
        member = self.members.get(member_id)
        book = self.books.get(book_id)
        if not member:
            print(f"Issue failed: member '{member_id}' not found.")
            return False
        if not book:
            print(f"Issue failed: book '{book_id}' not found.")
            return False
        success = member.issue_book(book)
        if success:
            print(f"Issued book [{book_id}] to {member.name}.")
        else:
            # give reason
            if not book.available:
                print(f"Issue failed: book [{book_id}] is already issued.")
            elif not member.can_borrow():
                print(f"Issue failed: {member.name} reached borrowing limit ({member.BORROW_LIMIT}).")
            else:
                print("Issue failed: unknown reason.")
        return success

    def return_book(self, member_id: str, book_id: str) -> bool:
        member = self.members.get(member_id)
        book = self.books.get(book_id)
        if not member or not book:
            print("Return failed: invalid member or book id.")
            return False
        success = member.return_book(book)
        if success:
            print(f"Book [{book_id}] returned by {member.name}.")
        else:
            print(f"Return failed: {member.name} does not have book [{book_id}].")
        return success

    def display_all_books(self):
        print("Library Books:")
        for book in self.books.values():
            book.display_info()

    def display_all_members(self):
        print("Library Members:")
        for m in self.members.values():
            m.display_member_info()
            print("-" * 30)


In [12]:
if __name__ == "__main__":
    lib = Library()

    def input_nonempty(prompt):
        while True:
            v = input(prompt).strip()
            if v:
                return v
            print("Input cannot be empty.")

    def create_book_interactive():
        book_id = input_nonempty("Enter book id: ")
        title = input_nonempty("Enter book title: ")
        author = input_nonempty("Enter author: ")
        book = Book(book_id, title, author)
        added = book_id not in lib.books
        lib.add_book(book)
        print("Book added." if added else "Book id existed — replaced entry.")

    def create_member_interactive():
        member_id = input_nonempty("Enter member id: ")
        name = input_nonempty("Enter member name: ")
        while True:
            t = input("Member type (S=Student / T=Teacher): ").strip().upper()
            if t == "S":
                member = StudentMember(member_id, name)
                break
            if t == "T":
                member = TeacherMember(member_id, name)
                break
            print("Enter S or T.")
        lib.register_member(member)
        print("Member registered.")

    def issue_interactive():
        member_id = input_nonempty("Enter member id: ")
        book_id = input_nonempty("Enter book id: ")
        if lib.issue_book(member_id, book_id):
            print(f"Issued book [{book_id}] to {lib.members[member_id].name}.")
        else:
            # give detailed reason
            member = lib.members.get(member_id)
            book = lib.books.get(book_id)
            if not member:
                print("Issue failed: member not found.")
            elif not book:
                print("Issue failed: book not found.")
            elif not book.available:
                print("Issue failed: book already issued.")
            elif not member.can_borrow():
                print(f"Issue failed: {member.name} reached borrow limit ({member.BORROW_LIMIT}).")
            else:
                print("Issue failed.")

    def return_interactive():
        member_id = input_nonempty("Enter member id: ")
        book_id = input_nonempty("Enter book id: ")
        if lib.return_book(member_id, book_id):
            print(f"Book [{book_id}] returned by {lib.members[member_id].name}.")
        else:
            print("Return failed. Check ids or whether the book was issued to this member.")

    menu = (
        "\nMenu:\n"
        "1) Add book\n"
        "2) Register member\n"
        "3) Issue book\n"
        "4) Return book\n"
        "5) Display all books\n"
        "6) Display all members\n"
        "7) Exit\n"
    )

    while True:
        print(menu)
        choice = input("Choose an option (1-7): ").strip()
        if choice == "1":
            create_book_interactive()
        elif choice == "2":
            create_member_interactive()
        elif choice == "3":
            issue_interactive()
        elif choice == "4":
            return_interactive()
        elif choice == "5":
            lib.display_all_books()
        elif choice == "6":
            lib.display_all_members()
        elif choice == "7":
            print("Exiting.")
            break
        else:
            print("Invalid choice. Enter 1-7.")


Menu:
1) Add book
2) Register member
3) Issue book
4) Return book
5) Display all books
6) Display all members
7) Exit

Book added.

Menu:
1) Add book
2) Register member
3) Issue book
4) Return book
5) Display all books
6) Display all members
7) Exit

Library Books:
[d3454] animations by powell by powell — Available

Menu:
1) Add book
2) Register member
3) Issue book
4) Return book
5) Display all books
6) Display all members
7) Exit

Issue failed: member '7' not found.
Issue failed: member not found.

Menu:
1) Add book
2) Register member
3) Issue book
4) Return book
5) Display all books
6) Display all members
7) Exit

Enter S or T.
Member registered.

Menu:
1) Add book
2) Register member
3) Issue book
4) Return book
5) Display all books
6) Display all members
7) Exit

Library Members:
Student Member — ID: m098, Name: mustabshirah ahmed
(Student borrow limit: 3)
Member ID: m098 | Name: mustabshirah ahmed
Issued books (0):
------------------------------

Menu:
1) Add book
2) Register mem